In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

In [ ]:
# 1. block mult case (ms)
blockmult_data_bs = {}
with open ('mac_bm.txt', 'r') as macbm:
    with open('gadi_bm.txt', 'r') as gadibm:
        while True:
            # read 
            macbm.readline()
            gadibm.readline()
            acc = (str(macbm.readline())[:-1]).split(":")
            if len(acc) != 3:
                print(acc)
                break
            dim, nt, bsize = acc
#             print(dim, nt, bsize)
            dim2, nt2, bsize2 = (str(gadibm.readline())[:-1]).split(":")
            if dim != dim2 or nt != nt2 or bsize != bsize2:
                print("error")
            if bsize not in blockmult_data_bs:
                blockmult_data_bs[bsize] = dict()
                blockmult_data_bs[bsize]['np'] = [1,2,4,8,16]

            if (str(dim) + 'mac1') not in blockmult_data_bs[bsize]:
                blockmult_data_bs[bsize][str(dim) + 'mac1'] = []
            if (str(dim) + 'mac2') not in blockmult_data_bs[bsize]:
                blockmult_data_bs[bsize][str(dim) + 'mac2'] = []
                
            macbm.readline() #v1
            macv1 = str(macbm.readline())
            macv1 = float(macv1[:-1]) if macv1[-1] == '\n' else float(macv1)
            blockmult_data_bs[bsize][str(dim) + 'mac1'].append(macv1)
            macbm.readline() #v2
            macv2 = str(macbm.readline())
            macv2 = float(macv2[:-1]) if macv2[-1] == '\n' else float(macv2)
            blockmult_data_bs[bsize][str(dim) + 'mac2'].append(macv2)

            if (str(dim) + 'gadi1') not in blockmult_data_bs[bsize]:
                blockmult_data_bs[bsize][str(dim) + 'gadi1'] = []
            if (str(dim) + 'gadi2') not in blockmult_data_bs[bsize]:
                blockmult_data_bs[bsize][str(dim) + 'gadi2'] = []
            
            gadibm.readline() #v1
            gadiv1 = str(gadibm.readline())
            gadiv1 = float(gadiv1[:-1]) if gadiv1[-1] == '\n' else float(gadiv1)
            blockmult_data_bs[bsize][str(dim) + 'gadi1'].append(gadiv1)
    
            gadibm.readline() #v2
            gadiv2 = str(gadibm.readline())
            gadiv2 = float(gadiv2[:-1]) if gadiv2[-1] == '\n' else float(gadiv2)
            blockmult_data_bs[bsize][str(dim) + 'gadi2'].append(gadiv2)
        
        gadibm.close()
    macbm.close()

blockmult_data_bs

In [ ]:
# part 2. summa mpi case

def getdict(filename1, filename2):
    summa_data_bs = {}
    summa_data_bs['np'] = [1,2,4,8,16]

    with open (filename1, 'r') as macsm:
        with open(filename2, 'r') as gadism:
            while True:
                # read 
                macsm.readline()
                gadism.readline()
                acc = (str(macsm.readline())[:-1]).split(":")
                if len(acc) != 2:
                    print(acc)
                    break
                dim, nt = acc
                dim2, nt2 = (str(gadism.readline())[:-1]).split(":")
                if dim != dim2 or nt != nt2:
                    print("error")

                if (str(dim) + 'mac1') not in summa_data_bs:
                    summa_data_bs[str(dim) + 'mac1'] = []
                if (str(dim) + 'mac2') not in summa_data_bs:
                    summa_data_bs[str(dim) + 'mac2'] = []

                macsm.readline() #v1
                macv1 = str(macsm.readline())
                macv1 = float(macv1[:-1]) if macv1[-1] == '\n' else float(macv1)
                summa_data_bs[str(dim) + 'mac1'].append(macv1)
                macsm.readline() #v2
                macv2 = str(macsm.readline())
                macv2 = float(macv2[:-1]) if macv2[-1] == '\n' else float(macv2)
                summa_data_bs[str(dim) + 'mac2'].append(macv2)

                if (str(dim) + 'gadi1') not in summa_data_bs:
                    summa_data_bs[str(dim) + 'gadi1'] = []
                if (str(dim) + 'gadi2') not in summa_data_bs:
                    summa_data_bs[str(dim) + 'gadi2'] = []

                gadism.readline() #v1
                gadiv1 = str(gadism.readline())
                gadiv1 = float(gadiv1[:-1]) if gadiv1[-1] == '\n' else float(gadiv1)
                summa_data_bs[str(dim) + 'gadi1'].append(gadiv1)

                gadism.readline() #v2
                gadiv2 = str(gadism.readline())
                gadiv2 = float(gadiv2[:-1]) if gadiv2[-1] == '\n' else float(gadiv2)
                summa_data_bs[str(dim) + 'gadi2'].append(gadiv2)

            gadism.close()
        macbm.close()

        return summa_data_bs

summa_data = getdict('mac_sm.txt', 'gadi_sm.txt')
cannon_data = getdict('mac_cn.txt', 'gadi_cn.txt')
summap_data = getdict('mac_smp.txt', 'gadi_smp.txt')
cannonp_data = getdict('mac_cnp.txt', 'gadi_cnp.txt')
ssort_data = getdict('mac_ss.txt', 'gadi_ss.txt')

In [ ]:
def part1Draw(data, taskname, bs=0):
    df = pd.DataFrame.from_dict(data)
    df.set_index('np')
    cols = df.columns[1:]
    i = 0
    j = 0
    fig, axes = plt.subplots(2, 2,  figsize=(16,16))
    colors = ['b','g','r','c','m','y','k']
    while i < len(cols):
        macv1 = cols[i]
        macv2 = cols[i+1]
        gadiv1 = cols[i+2]
        gadiv2 = cols[i+3]
        x = 1 if i > len(cols)/2 else 0
        ml1 = axes[x][0].plot(df['np'], df[macv1], '-ok', color=colors[j],label='MacOS version 1 {0}'.format(cols[i][:-4]))
        ml2 = axes[x][0].plot(df['np'], df[macv2], '-ok', color=colors[j+1],label='MacOS version 2 {0}'.format(cols[i+1][:-4]))
        gl1 = axes[x][1].plot(df['np'], df[gadiv1], '-ok', color=colors[j],label='Gadi version 1 {0}'.format(cols[i+2][:-5]))
        gl2 = axes[x][1].plot(df['np'], df[gadiv2], '-ok', color=colors[j+1],label='Gadi version 2 {0}'.format(cols[i+3][:-5]))
        axes[x][0].legend()
        axes[x][0].set_ylabel('ms')
        axes[x][1].legend()

        j = j + 2
        if j >= 6: 
            j = 0
        i = i + 4
    axes[-1][0].set_xlabel('Number of process/thread')
    axes[-1][1].set_xlabel('Number of process/thread')
    fig.suptitle('Scatter Line Plot - Running time vs Number of Process/Thread under various matrix dimension - {0} {1}'.format(taskname, "blocksize: " + str(bs) if int(bs) > 0 else ""), fontsize=16)
    fig.tight_layout()    
    fig.savefig("{0}{1}.jpg".format(taskname, bs if int(bs) > 0 else ""))

In [ ]:
# t1 blockmult
for bsize, bmult_data in blockmult_data_bs.items():
    for k, d in bmult_data.items():
        if k.count('1000'):
            print("bsize: ", bsize, k, d[-1])
#     part1Draw(bmult_data, 'Block Multiplication OpenMP', bsize)

# # t2 summa
# part1Draw(summa_data, 'SUMMA MPI')
# # t3 cannon
# part1Draw(cannon_data, 'Cannon MPI')
# # t4 summa pthread
# part1Draw(summap_data, 'SUMMA Pthread')
# # t4 cannon pthread
# part1Draw(cannonp_data, 'Cannon Pthread')
# pd.DataFrame.from_dict(summap_data)


In [ ]:
ssort_data = getdict('mac_ss.txt', 'gadi_ss.txt')
# part1Draw(ssort_data, "SSort MPI")
pd.DataFrame.from_dict(ssort_data)

In [ ]:
# plot speed up and efficiency
def S(Ts, Tp):
    return float(Ts/Tp)
def E(S, P):
    return float(S/P)

def computeSE(data):
    df = pd.DataFrame.from_dict(data)
    df.set_index('np')
    cols = df.columns[1:]
    Speedups = {}
    Speedups['np'] = list(df['np'])
    Efficiencies = {}
    Efficiencies['np'] = list(df['np'])
    for col in cols:
        base = df[col][0]
        procs = df['np']
        S_all = []
        E_all = []
        for i, p in enumerate(df['np']):
            s = S(base, df[col][i])
            e = E(s, procs[i])
            S_all.append(s)
            E_all.append(e)
#         print(col, S_all, E_all)
        Speedups[col] = S_all
        Efficiencies[col] = E_all
    return Speedups, Efficiencies
    
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
# {index, macv1S, macv1E, macv2S, macv2E, gadiv1S, gadiv1E, gadiv2S, gadiv2E}
def plotSE(final_s, final_e, taskname, bs=-1):
    cmap = get_cmap(10)
    sdf = pd.DataFrame.from_dict(final_s)
    edf = pd.DataFrame.from_dict(final_e)
    fig, axes = plt.subplots(1, 2,  figsize=(16,8))
    cols = ['500mac1', '500mac2', '1000mac1', '1000mac2', '500gadi1', '500gadi2', '1000gadi1', '1000gadi2']
    cole = ['500mac1', '500mac2', '1000mac1', '1000mac2', '500gadi1', '500gadi2', '1000gadi1', '1000gadi2']
    for i, col in enumerate(cols):
        label = ""
        if col.count('mac1'):
            label = "MacOS version 1"
        elif col.count('mac2'):
            label = "MacOS version 2"
        elif col.count('gadi1'):
            label = "Gadi version 1"
        elif col.count('gadi2'):
            label = "Gadi version 2"
        else:
            print("Error")
        label += " Dim=" + col[:4]
        line = axes[0].plot(sdf['np'], sdf[col], '-ok', color=cmap(i),label=label)
    axes[0].legend()
    axes[0].set_ylabel('Speed Up')

    for i, col in enumerate(cole):
        label = ""
        if col.count('mac1'):
            label = "MacOS version 1"
        elif col.count('mac2'):
            label = "MacOS version 2"
        elif col.count('gadi1'):
            label = "Gadi version 1"
        elif col.count('gadi2'):
            label = "Gadi version 2"
        else:
            print("Error")
        label += " Dim=" + col[:4]
        line = axes[1].plot(edf['np'], edf[col], '-ok', color=cmap(i),label=label)
    axes[1].legend()
    axes[1].set_ylabel('Efficiency')
    fig.suptitle("Scatter Line Plot - Speed Up vs Number of Process/Thread - {0} {1}".format(taskname, "blocksize: " + str(bs) if int(bs) > 0 else ""), fontsize=16)
    fig.tight_layout()    
    fig.savefig("SE{0}{1}.jpg".format(taskname, bs if int(bs) > 0 else ""))
    
        

# t1 blockmult
# for bsize, bmult_data in blockmult_data_bs.items():
#     final_s, final_e = computeSE(bmult_data)
pd.DataFrame.from_dict(computeSE(blockmult_data_bs['48'])[1])
#     pd.DataFrame.from_dict(final_e)
#     plotSE(final_s, final_e, "Block Multiplication OpenMP", bsize)

# t2 summa
# final_s, final_e = computeSE(summa_data)
# pd.DataFrame.from_dict(final_e)
# plotSE(final_s, final_e, "SUMMA MPI", 0)
# # t3 cannon
# final_s, final_e = computeSE(cannon_data)
# pd.DataFrame.from_dict(final_e)
# plotSE(final_s, final_e, "Cannon MPI", 0)
# # t4 summa pthread
# final_s, final_e = computeSE(summap_data)
# pd.DataFrame.from_dict(final_e)
# plotSE(final_s, final_e, "SUMMA Pthread", 0)
# # t4 cannon pthread
# final_s, final_e = computeSE(cannonp_data)
# plotSE(final_s, final_e, "Cannon Pthread", 0)

# final_s, final_e = computeSE(ssort_data)
# pd.DataFrame.from_dict(final_e)
# plotSE(final_s, final_e, "SSort MPI", 0)